In [1]:
import pandas as pd
import os
%pwd

u'/Users/Driz/data_sci_wk3'

In [4]:
df = pd.read_csv('../DSI-CHI-1/lessons/week-06/3.1-random-forests-boosting/assets/datasets/car.csv')

In [5]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [6]:
for column in df.columns:
    print df[column].describe()

count     1728
unique       4
top        med
freq       432
Name: buying, dtype: object
count     1728
unique       4
top        med
freq       432
Name: maint, dtype: object
count     1728
unique       4
top          3
freq       432
Name: doors, dtype: object
count     1728
unique       3
top       more
freq       576
Name: persons, dtype: object
count     1728
unique       3
top        med
freq       576
Name: lug_boot, dtype: object
count     1728
unique       3
top        med
freq       576
Name: safety, dtype: object
count      1728
unique        4
top       unacc
freq       1210
Name: acceptability, dtype: object


In [7]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1))

In [8]:
y

array([2, 2, 2, ..., 2, 1, 3])

In [13]:
# shows how unbalanced it is 
pd.DataFrame(y)[0].value_counts()

2    1210
0     384
1      69
3      65
Name: 0, dtype: int64

In [10]:
X.head()

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [11]:
X.columns

Index([u'buying_high', u'buying_low', u'buying_med', u'buying_vhigh',
       u'maint_high', u'maint_low', u'maint_med', u'maint_vhigh', u'doors_2',
       u'doors_3', u'doors_4', u'doors_5more', u'persons_2', u'persons_4',
       u'persons_more', u'lug_boot_big', u'lug_boot_med', u'lug_boot_small',
       u'safety_high', u'safety_low', u'safety_med'],
      dtype='object')

In [14]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)

In [15]:
cv

sklearn.cross_validation.StratifiedKFold(labels=[2 2 2 ..., 2 1 3], n_folds=3, shuffle=True, random_state=41)

In [16]:
# makes models in each fold and gives score for each run--CV
dt = DecisionTreeClassifier(class_weight='balanced')
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)

In [17]:
# this is confusing
print "{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s.mean().round(3), s.std().round(3))

Decision Tree Score:	0.965 ± 0.008


In [19]:
def score(model, name):
    s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [20]:
bdt = BaggingClassifier(DecisionTreeClassifier())

In [21]:
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

In [22]:
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

In [23]:
score(dt, "Decision Tree")

Decision Tree Score:	0.965 ± 0.008


In [24]:
score(bdt, "Bagging DT")

Bagging DT Score:	0.972 ± 0.014


In [25]:
score(rf, "Random Forest")

Random Forest Score:	0.948 ± 0.009


In [26]:
score(et, "Extra Trees")

Extra Trees Score:	0.951 ± 0.011


In [27]:
et.fit(X, y)

ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [28]:
predictions = et.predict(X)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix

print confusion_matrix(y, predictions)

[[ 384    0    0    0]
 [   0   69    0    0]
 [   0    0 1210    0]
 [   0    0    0   65]]


In [30]:
print classification_report(y,predictions)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       384
          1       1.00      1.00      1.00        69
          2       1.00      1.00      1.00      1210
          3       1.00      1.00      1.00        65

avg / total       1.00      1.00      1.00      1728



Holy Moly, thats overfit!

lets do some boosting!

In [31]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
ab = AdaBoostClassifier()
gb = GradientBoostingClassifier()

# AdaBoost Score:   0.811 ± 0.002
# GBoost Score: 0.982 ± 0.006

In [32]:
# still using score function from above
score(ab, "AdaBoost")

AdaBoost Score:	0.811 ± 0.002


In [33]:
# much slower. one tree at a time
score(gb, "Gradient Boosting Classifier")

Gradient Boosting Classifier Score:	0.982 ± 0.006
